In [1]:
import os
import sys
import json
import string

import collections 
import pandas as pd

from dotenv import load_dotenv

In [2]:
load_dotenv()

path = os.environ.get("PATH_SPIDER")
print(os.listdir(path))


['dev.json', 'README.txt', 'train_gold.sql', '.DS_Store', 'train_spider.json', 'tables.json', 'dev_gold.sql', 'train_others.json', 'database']


In [3]:
def sql2df(queri_name):
    with open(os.path.join(path, queri_name), 'r') as f:
        lines = f.readlines()
    
    data = collections.defaultdict(list)
    for line in lines:
        line_split = line.split('\t')
        data['queri'].append(line_split[0])
        data['ids'].append(line_split[-1].split('\n')[0])
    return pd.DataFrame(data)
        

In [4]:
train_gold_df = sql2df('train_gold.sql')
dev_gold_df = sql2df('dev_gold.sql')

In [5]:
print(train_gold_df.shape)
print(dev_gold_df.shape)

(8659, 2)
(1034, 2)


In [24]:
def mergeJson(file):
    merge = []
    for f in file:
        with open(f,'r') as file:
            merge.extend(json.load(file))
            
    with open('train_collacted.json', 'w') as out:
        json.dump(merge, out)
        
        
def filter_query_toks(query_toks):
    dummy = []
    for x in query_toks:
        if x in ["''", "``", ';', '.']:
            pass
        else:
            dummy.append(x.lower())
    return dummy

def masking_query_toks(query_toks):
    dummy = []
    for x in query_toks:
        if x in ['value']:
            dummy.append("[MASK]")
        else:
            dummy.append(x)    
    return dummy

def get_value(query_toks, query_toks_no_value):
    dummy = []
    # query_toks = filter_query_toks(query_toks)
    for x, y in zip(query_toks, query_toks_no_value):
        if y == 'value':
            dummy.append(x)
    return dummy

def query_tokens(query):
    tokens = []
    for toks in query.replace(";","").split():
        if "." in toks:
            try:
                tokens.append(str(float(toks)).lower())
            except:
                index = toks.find(".")
                tokens.extend([str(toks[:index]).lower(), str(toks[index]).lower(), str(toks[index+1:]).lower()])
        else:
            tokens.append(toks.lower())
            
    return tokens


def query_tokens_v2(query):
    tokens = []
    for toks in query.replace(";", "").split():
        if "." in toks:
            try:
                tokens.append(str(float(toks)))
            except:
                index = toks.find(".")
                tokens.extend([str(toks[:index]), str(toks[index]), str(toks[index+1:])])
        else:
            tokens.append(toks)
    return tokens

def complete_query_toks(query_toks):
    dummy = []
    query_toks = query_tokens(query_toks)
    for i, l in enumerate(query_toks):
        if '"' in l:
            try:
                if '"' in query_toks[1+i]:
                    dummy.append(l + " " + query_toks[i+1])
                else:        
                    if l.find('"') < 2:
                        dummy.append(l)
            except:
                continue        
        else:
            dummy.append(l)      
    return dummy

files = [os.path.join(path, 'train_spider.json'), os.path.join(path, 'train_others.json')]
mergeJson(files)

In [7]:
train_collacted = open('train_collacted.json')
train_collacted_file = json.loads(train_collacted.read())

spider_json_file = open(os.path.join(path, 'train_spider.json'))
others_json_file = open(os.path.join(path, 'train_others.json'))
train_spider_json = json.loads(spider_json_file.read())
others_json = json.loads(others_json_file.read())

In [8]:
def json2df(json_file):
    collect = collections.defaultdict(list)
    
    for data in json_file:
        collect['ids'].append(data['db_id'])
        collect['query'].append(data['query'])
        collect['question'].append(data['question'])
        collect['query_toks'].append(complete_query_toks(data['query']))
        collect['query_toks_no_value'].append(data['query_toks_no_value'])
        collect['mask_query'].append(masking_query_toks(data['query_toks_no_value']))
        collect['value'].append(get_value(complete_query_toks(data['query']), data['query_toks_no_value']))
    
    return pd.DataFrame(collect)

In [9]:
other_df = json2df(others_json)

In [10]:
other_df.tail()

,ids,query,question,query_toks,query_toks_no_value,mask_query,value
1654,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...",where can i find a restaurant in the bay area ?,"[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[""bay area""]"
1655,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...",give me a good restaurant in the bay area ?,"[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[""region0"", 2.5]"
1656,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...",what are some good restaurants in the bay area ?,"[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[""region0"", 2.5]"
1657,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...",give me some good restaurants in the bay area ?,"[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[""region0"", 2.5]"
1658,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...",what is a good restaurant in the bay area ?,"[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[select, t2, ., house_number, ,, t1, ., name, ...","[""region0"", 2.5]"


In [11]:
other_df.iloc[75]

ids                                                                  geo
query                  SELECT river_name FROM river WHERE LENGTH  =  ...
question                 what is the longest river flowing through texas
query_toks             [select, river_name, from, river, where, lengt...
query_toks_no_value    [select, river_name, from, river, where, lengt...
mask_query             [select, river_name, from, river, where, lengt...
value                                                          ["texas"]
Name: 75, dtype: object

In [12]:
len(complete_query_toks(others_json[75]['query']))

23

In [13]:
print(others_json[75]['query_toks_no_value'])
print(others_json[75]['query'])

['select', 'river_name', 'from', 'river', 'where', 'length', '=', '(', 'select', 'max', '(', 'length', ')', 'from', 'river', 'where', 'traverse', '=', 'value', ')', 'and', 'traverse', '=', 'value']
SELECT river_name FROM river WHERE LENGTH  =  ( SELECT MAX ( LENGTH ) FROM river WHERE traverse  =  "texas" ) AND traverse  =  "texas";


In [14]:
get_value(complete_query_toks(others_json[1624]['query']), others_json[1624]['query_toks_no_value'])

                

['"yosemite', '.', '.']

In [15]:
# print(complete_query_toks(others_json[1624]['query']))
# print(others_json[1624]['query_toks_no_value'])

# for a, b in zip (complete_query_toks(others_json[1624]['query']), others_json[1624]['query_toks_no_value']):
#    print(a, b)

print(others_json[1624]['query'])
print(others_json[1624]['query_toks_no_value'])

SELECT t3.house_number  ,  t1.name FROM restaurant AS t1 JOIN geographic AS t2 ON t1.city_name  =  t2.city_name JOIN LOCATION AS t3 ON t1.id  =  t3.restaurant_id WHERE t2.region  =  "yosemite and mono lake area" AND t1.food_type  =  "french" AND t1.rating  >  2.5;
['select', 't3', '.', 'house_number', ',', 't1', '.', 'name', 'from', 'restaurant', 'as', 't1', 'join', 'geographic', 'as', 't2', 'on', 't1', '.', 'city_name', '=', 't2', '.', 'city_name', 'join', 'location', 'as', 't3', 'on', 't1', '.', 'id', '=', 't3', '.', 'restaurant_id', 'where', 't2', '.', 'region', '=', 'value', 'and', 't1', '.', 'food_type', '=', 'value', 'and', 't1', '.', 'rating', '>', 'value']


In [16]:
query_list = query_tokens(others_json[1624]['query'])
queries = {}
for i, l in enumerate(query_list):
    queries[i] = l.find('"')
    
# query_tokens(others_json[1624]['query'])[41:46]

In [17]:
def complete_query_toks(query_toks):
    dummy = []
    query_toks = query_tokens(query_toks)
    for i, l in enumerate(query_toks):
        if '"' in l:
            try:
                if '"' in query_toks[1+i]:
                    dummy.append(l + " " + query_toks[i+1])
                else:        
                    if l.find('"') < 2:
                        dummy.append(l)
            except:
                continue        
        else:
            dummy.append(l)      
    return dummy

In [45]:
def complete_query_toks_v2(query_toks):
    dummy = []
    listed = []
    query_toks = query_tokens(query_toks)
    for i, l in enumerate(query_toks):
        if '"' in l:
            try:
                if l.find('"') == 0:
                    index = [x.find('"') for x in query_toks[i:]]
                    maks = max(index)
                    listed = query_toks[i:i+maks+1]
                    dummy.append(" ".join(listed))
                    
            except:
                pass
        else:
            if l in listed:
                pass
            else:
                dummy.append(l)
                
    return dummy


def complete_query_toks_v3(query_toks):
    dummy, listed = [], []
    query_toks = query_tokens_v2(query_toks)
    for i, l in enumerate(query_toks):
        if '"' in l:
            try:
                if l.find('"') == 0:
                    index = [x.find('"') for x in query_toks[i:]]
                    maks = max(index)
                    listed = query_toks[i:i+maks+1]
                    dummy.append(" ".join(listed))
            except:
                pass
        else:
            if l in listed:
                pass
            else:
                dummy.append(l)
                
    return dummy


print(len(complete_query_toks_v2(others_json[1624]['query'])))
print(len(others_json[1624]['query_toks_no_value']))

53
54


In [46]:
query_toks_v2 = query_tokens_v2(others_json[1624]['query'])
print(len(complete_query_toks_v3(others_json[1624]['query'])))

54


In [47]:
def json2df_v2(json_file):
    collect = collections.defaultdict(list)
    
    for data in json_file:
        collect['ids'].append(data['db_id'])
        collect['query'].append(data['query'])
        collect['question'].append(data['question'])
        collect['query_toks'].append(complete_query_toks_v3(data['query']))
        collect['query_toks_no_value'].append(data['query_toks_no_value'])
        collect['mask_query'].append(masking_query_toks(data['query_toks_no_value']))
        collect['value'].append(get_value(complete_query_toks_v3(data['query']), data['query_toks_no_value']))
    
    return pd.DataFrame(collect)

other_df_v2 = json2df_v2(others_json)


In [54]:
pd.set_option('display.max_colwidth', None)
other_df_v2[['query_toks', 'query_toks_no_value', 'value']].tail(50)

,query_toks,query_toks_no_value,value
1609,"[SELECT, t3, ., house_number, ,, t1, ., name, FROM, restaurant, AS, t1, JOIN, geographic, AS, t2, ON, t1, ., city_name, =, t2, ., city_name, JOIN, LOCATION, AS, t3, ON, t1, ., id, =, t3, ., restaurant_id, WHERE, t2, ., region, =, ""bay area"" AND t1 ., food_type, =, ""american"" AND t1 . rating, =, (, SELECT, MAX, (, ), FROM, restaurant, AS, JOIN, geographic, AS, t2, ON, city_name, =, t2, city_name, WHERE, t2, region, =, ""bay area"" AND t1 ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, value, and, t1, ., food_type, =, value, )]","[""bay area"" AND t1 ., SELECT]"
1610,"[SELECT, t3, ., house_number, ,, t1, ., name, FROM, restaurant, AS, t1, JOIN, geographic, AS, t2, ON, t1, ., city_name, =, t2, ., city_name, JOIN, LOCATION, AS, t3, ON, t1, ., id, =, t3, ., restaurant_id, WHERE, t2, ., region, =, ""bay area"" AND t1 ., food_type, =, ""american"" AND t1 . rating, =, (, SELECT, MAX, (, ), FROM, restaurant, AS, JOIN, geographic, AS, t2, ON, city_name, =, t2, city_name, WHERE, t2, region, =, ""bay area"" AND t1 ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, value, and, t1, ., food_type, =, value, )]","[""bay area"" AND t1 ., SELECT]"
1611,"[SELECT, t3, ., house_number, ,, t1, ., name, FROM, restaurant, AS, t1, JOIN, geographic, AS, t2, ON, t1, ., city_name, =, t2, ., city_name, JOIN, LOCATION, AS, t3, ON, t1, ., id, =, t3, ., restaurant_id, WHERE, t2, ., region, =, ""bay area"" AND t1 ., food_type, =, ""american"" AND t1 . rating, =, (, SELECT, MAX, (, ), FROM, restaurant, AS, JOIN, geographic, AS, t2, ON, city_name, =, t2, city_name, WHERE, t2, region, =, ""bay area"" AND t1 ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, value, and, t1, ., food_type, =, value, )]","[""bay area"" AND t1 ., SELECT]"
1612,"[SELECT, t3, ., house_number, ,, t1, ., name, FROM, restaurant, AS, t1, JOIN, geographic, AS, t2, ON, t1, ., city_name, =, t2, ., city_name, JOIN, LOCATION, AS, t3, ON, t1, ., id, =, t3, ., restaurant_id, WHERE, t2, ., region, =, ""bay area"" AND t1 ., food_type, =, ""american"" AND t1 . rating, =, (, SELECT, MAX, (, ), FROM, restaurant, AS, JOIN, geographic, AS, t2, ON, city_name, =, t2, city_name, WHERE, t2, region, =, ""bay area"" AND t1 ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where

In [ ]:
others_json[1609]

In [21]:
for a, b in zip(complete_query_toks_v2(others_json[1624]['query']), others_json[1624]['query_toks_no_value']):
    print(a, b)

select select
t3 t3
. .
house_number house_number
, ,
t1 t1
. .
name name
from from
restaurant restaurant
as as
t1 t1
join join
geographic geographic
as as
t2 t2
on on
t1 t1
. .
city_name city_name
= =
t2 t2
. .
city_name city_name
join join
location location
as as
t3 t3
on on
t1 t1
. .
id id
= =
t3 t3
. .
restaurant_id restaurant_id
where where
t2 t2
. .
region region
= =
"yosemite and mono lake area" value
t1 and
. t1
food_type .
= food_type
"french" =
and value
t1 and
. t1
rating .
> rating
2.5 >


In [22]:
others_json[1624]['query']

'SELECT t3.house_number  ,  t1.name FROM restaurant AS t1 JOIN geographic AS t2 ON t1.city_name  =  t2.city_name JOIN LOCATION AS t3 ON t1.id  =  t3.restaurant_id WHERE t2.region  =  "yosemite and mono lake area" AND t1.food_type  =  "french" AND t1.rating  >  2.5;'

In [19]:
dummy = []

mins = 0
maks = len(query_list)
listed = []
for i, l in enumerate(query_list):
    if '"' in l:
        try:
            if l.find('"') == 0:
                index = [x.find('"') for x in query_list[i:]]
                maks = max(index)
                listed = query_list[i:i+maks+1]
                print(" ".join(query_list[i:i+maks+1]))
                
        except:
            continue          
    else:
        if l in listed:
            continue
        else:
            print(l)
        

select
t3
.
house_number
,
t1
.
name
from
restaurant
as
t1
join
geographic
as
t2
on
t1
.
city_name
=
t2
.
city_name
join
location
as
t3
on
t1
.
id
=
t3
.
restaurant_id
where
t2
.
region
=
"yosemite and mono lake area"
t1
.
food_type
=
"french"
and
t1
.
rating
>
2.5


In [126]:
index = [x.find('"') for x in query_list[41:]]
mins = min(index)
maks = max(index)

In [125]:
index[]

4

In [40]:
pd.set_option('display.max_colwidth', None)
other_df[['query_toks', 'query_toks_no_value','value']].tail(50)

,query_toks,query_toks_no_value,value
1609,"[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, value, and, t1, ., food_type, =, value, )]","[""bay area"", ""american"", ""bay area"", ""american""]"
1610,"[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, value, and, t1, ., food_type, =, value, )]","[""bay area"", ""american"", ""bay area"", ""american""]"
1611,"[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, value, and, t1, ., food_type, =, value, and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, value, and, t1, ., food_type, =, value, )]","[""bay area"", ""american"", ""bay area"", ""american""]"
1612,"[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, ., restaurant_id, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", and, t1, ., rating, =, (, select, max, (, t1, ., rating, ), from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, where, t2, ., region, =, ""bay area"", and, t1, ., food_type, =, ""american"", )]","[select, t3, ., house_number, ,, t1, ., name, from, restaurant, as, t1, join, geographic, as, t2, on, t1, ., city_name, =, t2, ., city_name, join, location, as, t3, on, t1, ., id, =, t3, 

In [14]:
others_json[-2]['query_toks_no_value']

['select',
 't2',
 '.',
 'house_number',
 ',',
 't1',
 '.',
 'name',
 'from',
 'location',
 'as',
 't2',
 'join',
 'restaurant',
 'as',
 't1',
 'on',
 't1',
 '.',
 'id',
 '=',
 't2',
 '.',
 'restaurant_id',
 'where',
 't1',
 '.',
 'city_name',
 'in',
 '(',
 'select',
 'city_name',
 'from',
 'geographic',
 'where',
 'region',
 '=',
 'value',
 ')',
 'and',
 't1',
 '.',
 'rating',
 '>',
 'value']

In [54]:
print(other_df.iloc[-1]['query_toks'])
print(other_df.iloc[-1]['query_toks_no_value'])

for a, b in zip(other_df.iloc[-1]['query_toks'], other_df.iloc[-1]['query_toks_no_value']):
    if b == 'value':
        print(a)

['select', 't2', '.', 'house_number', ',', 't1', '.', 'name', 'from', 'location', 'as', 't2', 'join', 'restaurant', 'as', 't1', 'on', 't1', '.', 'id', '=', 't2', '.', 'restaurant_id', 'where', 't1', '.', 'city_name', 'in', '(', 'select', 'city_name', 'from', 'geographic', 'where', 'region', '=', 'region0', ')', 'and', 't1', '.', 'rating', '>', '2.5']
['select', 't2', '.', 'house_number', ',', 't1', '.', 'name', 'from', 'location', 'as', 't2', 'join', 'restaurant', 'as', 't1', 'on', 't1', '.', 'id', '=', 't2', '.', 'restaurant_id', 'where', 't1', '.', 'city_name', 'in', '(', 'select', 'city_name', 'from', 'geographic', 'where', 'region', '=', 'value', ')', 'and', 't1', '.', 'rating', '>', 'value']
region0
2.5


In [16]:
all_lines = '<html><head>'
d = ">"
for line in all_lines:
    s =  [e+d for e in line.split(d) if e]

In [72]:
s = [e+d for e in all_lines.split(d) if e]

In [74]:
for e in all_lines.split(d):
    print(e)

<html
<head



In [245]:
tokens = []
for toks in others_json[1654]['query'].replace(";", "").split('" "'):
    if "." in toks:
        try:
            tokens.append(str(float(toks)))
        except:
            index = toks.find(".")
            tokens.extend([str(toks[:index]), str(toks[index]), str(toks[index+1:])])      
    else:
        tokens.append(toks)
        
tokens


     

['SELECT t2',
 '.',
 'house_number  ,  t1.name FROM LOCATION AS t2 JOIN restaurant AS t1 ON t1.id  =  t2.restaurant_id WHERE t1.city_name IN ( SELECT city_name FROM geographic WHERE region  =  "bay area" )']

In [255]:
others_json[1654]['query'].replace(";", "").split('"')

['SELECT t2.house_number  ,  t1.name FROM LOCATION AS t2 JOIN restaurant AS t1 ON t1.id  =  t2.restaurant_id WHERE t1.city_name IN ( SELECT city_name FROM geographic WHERE region  =  ',
 'bay area',
 ' )']

In [258]:
" ".join(others_json[1654]['query_toks_no_value'])

'select t2 . house_number , t1 . name from location as t2 join restaurant as t1 on t1 . id = t2 . restaurant_id where t1 . city_name in ( select city_name from geographic where region = value )'

In [259]:
others_json[1654]['query']

'SELECT t2.house_number  ,  t1.name FROM LOCATION AS t2 JOIN restaurant AS t1 ON t1.id  =  t2.restaurant_id WHERE t1.city_name IN ( SELECT city_name FROM geographic WHERE region  =  "bay area" );'

In [262]:
''.join([('' if c in string.punctuation else " ")+c for c in others_json[1654]['query_toks_no_value']]).strip()

'select t2. house_number, t1. name from location as t2 join restaurant as t1 on t1. id= t2. restaurant_id where t1. city_name in( select city_name from geographic where region= value)'

In [275]:
index_titik = []
for i, l in enumerate(others_json[1654]['query_toks_no_value']):
    if l in ".":
        index_titik.append(i)
        


In [276]:
index_titik

[2, 6, 18, 22, 26]

In [9]:
train_spider_json

[{'db_id': 'department_management',
  'query': 'SELECT count(*) FROM head WHERE age  >  56',
  'query_toks': ['SELECT',
   'count',
   '(',
   '*',
   ')',
   'FROM',
   'head',
   'WHERE',
   'age',
   '>',
   '56'],
  'query_toks_no_value': ['select',
   'count',
   '(',
   '*',
   ')',
   'from',
   'head',
   'where',
   'age',
   '>',
   'value'],
  'question': 'How many heads of the departments are older than 56 ?',
  'question_toks': ['How',
   'many',
   'heads',
   'of',
   'the',
   'departments',
   'are',
   'older',
   'than',
   '56',
   '?'],
  'sql': {'from': {'table_units': [['table_unit', 1]], 'conds': []},
   'select': [False, [[3, [0, [0, 0, False], None]]]],
   'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]],
   'groupBy': [],
   'having': [],
   'orderBy': [],
   'limit': None,
   'intersect': None,
   'union': None,
   'except': None}},
 {'db_id': 'department_management',
  'query': 'SELECT name ,  born_state ,  age FROM head ORDER BY age',
  'query_

In [38]:
spider_df = pd.DataFrame(spider_collections)
spider_df.head()

,ids,query,query_no_value,question
0,department_management,SELECT count(*) FROM head WHERE age > 56,select count ( * ) from head where age > value,How many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","select name , born_state , age from head order...","List the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","select creation , name , budget_in_billions fr...","List the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","select max ( budget_in_billions ) , min ( budg...",What are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,select avg ( num_employees ) from department w...,What is the average number of employees of the...


In [18]:
train_spider_json[1]['sql']

{'from': {'table_units': [['table_unit', 1]], 'conds': []},
 'select': [False,
  [[0, [0, [0, 8, False], None]],
   [0, [0, [0, 9, False], None]],
   [0, [0, [0, 10, False], None]]]],
 'where': [],
 'groupBy': [],
 'having': [],
 'orderBy': ['asc', [[0, [0, 10, False], None]]],
 'limit': None,
 'intersect': None,
 'union': None,
 'except': None}

In [30]:
" ".join(map(train_spider_json[0]['query_toks_no_value']))

TypeError: map() must have at least two arguments.

In [17]:
dummy = []
for x in others_json[0]['query_toks']:
    if x in ["''","``"]:
        pass
    else:
        dummy.append(x)

In [18]:
for a, b in zip(dummy, others_json[0]['query_toks_no_value']):
    print(a, b)

SELECT select
city_name city_name
FROM from
city city
WHERE where
population population
= =
( (
SELECT select
MAX max
( (
population population
) )
FROM from
city city
WHERE where
state_name state_name
= =
wyoming value
) )
AND and
state_name state_name
= =
wyoming value


In [125]:
others_json[10]['query_toks_no_value']

['select',
 'city_name',
 'from',
 'city',
 'where',
 'population',
 '=',
 '(',
 'select',
 'max',
 '(',
 'population',
 ')',
 'from',
 'city',
 'where',
 'state_name',
 '=',
 'value',
 ')',
 'and',
 'state_name',
 '=',
 'value']

False